In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read Data
data = pd.read_csv("data.csv", header = None)

In [3]:
from keras.models import Sequential
from keras.layers import Dense
np.random.seed(7)

Using TensorFlow backend.


In [4]:
data.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
X=data.values[:,0:8]
Y=data.values[:,8]

In [7]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [9]:
model.fit(X,Y,epochs=150, batch_size=10)

Epoch 1/150
768/768 [==============================] - 2s - loss: 3.7276 - acc: 0.5990     
Epoch 2/150
768/768 [==============================] - 0s - loss: 0.9376 - acc: 0.5964     
Epoch 3/150
768/768 [==============================] - 0s - loss: 0.7488 - acc: 0.6484     
Epoch 4/150
768/768 [==============================] - 0s - loss: 0.7129 - acc: 0.6523     
Epoch 5/150
768/768 [==============================] - 0s - loss: 0.6850 - acc: 0.6667     
Epoch 6/150
768/768 [==============================] - 0s - loss: 0.6538 - acc: 0.6784     
Epoch 7/150
768/768 [==============================] - 0s - loss: 0.6538 - acc: 0.6784     
Epoch 8/150
768/768 [==============================] - 0s - loss: 0.6427 - acc: 0.6849     
Epoch 9/150
768/768 [==============================] - 0s - loss: 0.6291 - acc: 0.6927     
Epoch 10/150
768/768 [==============================] - 0s - loss: 0.6358 - acc: 0.6706     
Epoch 11/150
768/768 [==============================] - 0s - loss: 0.6537 - acc

768/768 [==============================] - 0s - loss: 0.5010 - acc: 0.7773     
Epoch 90/150
768/768 [==============================] - 0s - loss: 0.5092 - acc: 0.7565     
Epoch 91/150
768/768 [==============================] - 0s - loss: 0.4980 - acc: 0.7565     
Epoch 92/150
768/768 [==============================] - 0s - loss: 0.5113 - acc: 0.7487     
Epoch 93/150
768/768 [==============================] - 0s - loss: 0.4987 - acc: 0.7682     
Epoch 94/150
768/768 [==============================] - 0s - loss: 0.4962 - acc: 0.7656     
Epoch 95/150
768/768 [==============================] - 0s - loss: 0.5029 - acc: 0.7513     
Epoch 96/150
768/768 [==============================] - 0s - loss: 0.4905 - acc: 0.7669     
Epoch 97/150
768/768 [==============================] - 0s - loss: 0.4992 - acc: 0.7721     
Epoch 98/150
768/768 [==============================] - 0s - loss: 0.4888 - acc: 0.7682     
Epoch 99/150
768/768 [==============================] - 0s - loss: 0.4893 - acc: 0.

In [10]:
scores = model.evaluate(X,Y)

 32/768 [>.............................] - ETA: 0s

In [12]:
print(model.metrics_names[1], scores[1])

acc 0.794270833333


### K-fold Cross Verification

In [13]:
from sklearn.model_selection import StratifiedKFold


In [16]:
kfold = StratifiedKFold(n_splits=10, shuffle = True, random_state = 7)
cvscores=[]
for train,test in kfold.split(X,Y):
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=80, batch_size=10, verbose=0)
    #evaluate the model
    scores = model.evaluate(X[test],Y[test], verbose=0)
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1]*100)

print(np.mean(cvscores))

acc: 71.43%
acc: 70.13%
acc: 76.62%
acc: 71.43%
acc: 79.22%
acc: 71.43%
acc: 72.73%
acc: 63.64%
acc: 69.74%
acc: 65.79%
71.2149700003


### Using Keras with Scikit-Learn

In [17]:
## Using Built in Function build_fn

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score


In [19]:
## Function to create model, required for Keras Classifier
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

np.random.seed(7)

In [20]:
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

In [23]:
#Evaluate using KFold CV
kfold=StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
results=cross_val_score(model, X,Y, cv=kfold)

## Grid Search Deep Learning Model Parameters

In [24]:
from sklearn.model_selection import GridSearchCV

In [41]:
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation = 'relu'))
    model.add(Dense(1,activation ='sigmoid', kernel_initializer=init))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

np.random.seed(7)


In [42]:
model = KerasClassifier(build_fn = create_model, verbose=0)


In [43]:
#Grid Search Epochs, batch size and optimizer
optimizers=['rmsprop','adam']
inits = ['glorot_uniform', 'normal','uniform']
epochs=[50,100,150]
batches=[5,10,20]
param_grid = dict(optimizer = optimizers, epochs=epochs, init=inits)
grid = GridSearchCV(estimator=model, param_grid = param_grid)

In [44]:
grid_result = grid.fit(X,Y)

In [46]:
#Summarize Results:
print("Best: %f using %s" %(grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stds, params in zip(means, stds, params):
    print ("%f (%f) with %r" %(mean, stds,params))

Best: 0.734375 using {'epochs': 150, 'optimizer': 'adam', 'init': 'uniform'}
0.662760 (0.033197) with {'epochs': 50, 'optimizer': 'rmsprop', 'init': 'glorot_uniform'}
0.653646 (0.027498) with {'epochs': 50, 'optimizer': 'adam', 'init': 'glorot_uniform'}
0.695312 (0.005524) with {'epochs': 50, 'optimizer': 'rmsprop', 'init': 'normal'}
0.691406 (0.014616) with {'epochs': 50, 'optimizer': 'adam', 'init': 'normal'}
0.675781 (0.016877) with {'epochs': 50, 'optimizer': 'rmsprop', 'init': 'uniform'}
0.670573 (0.023939) with {'epochs': 50, 'optimizer': 'adam', 'init': 'uniform'}
0.670573 (0.016367) with {'epochs': 100, 'optimizer': 'rmsprop', 'init': 'glorot_uniform'}
0.661458 (0.035849) with {'epochs': 100, 'optimizer': 'adam', 'init': 'glorot_uniform'}
0.710938 (0.019137) with {'epochs': 100, 'optimizer': 'rmsprop', 'init': 'normal'}
0.710938 (0.013902) with {'epochs': 100, 'optimizer': 'adam', 'init': 'normal'}
0.688802 (0.016367) with {'epochs': 100, 'optimizer': 'rmsprop', 'init': 'unifor